In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python mediapipe numpy Pillow tqdm av torch torchvision pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --force-reinstall

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.6 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
import mediapipe as mp

# === PATHS ===
BASE_PATH = "/content/drive/MyDrive/FINAL DATASET"
CSV_OUTPUT_PATH = "/content/drive/MyDrive/FINAL DATASET/frame_level_feature_data.csv"

# === Setup Models ===
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(static_image_mode=False)
yolo = YOLO("yolov8n.pt")

# === Feature Extraction ===
frame_level_data = []

def process_video(video_path, category):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_area = frame_width * frame_height
    video_name = os.path.basename(video_path)

    frame_index = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # --- YOLO Person Detection ---
        results = yolo.predict(frame, verbose=False)[0]
        person_boxes = [b for b in results.boxes if int(b.cls[0]) == 0]
        if person_boxes:
            x1, y1, x2, y2 = person_boxes[0].xyxy[0].cpu().numpy()
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            bbox_area = (x2 - x1) * (y2 - y1)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            distance_to_center = np.linalg.norm([center_x - frame_width // 2, center_y - frame_height // 2])
        else:
            bbox_area = 0
            center_x = center_y = frame_width // 2
            distance_to_center = 0

        # --- Pose Keypoints ---
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose_model.process(frame_rgb)

        keypoints = []
        if result.pose_landmarks:
            for lm in result.pose_landmarks.landmark:
                x = int(lm.x * frame_width)
                y = int(lm.y * frame_height)
                keypoints.extend([x, y])
        else:
            keypoints = [0] * 66  # 33 keypoints × 2

        frame_level_data.append({
            "category": category,
            "video_name": video_name,
            "frame": frame_index,
            "frame_area": frame_area,
            "frame_width": frame_width,
            "frame_height": frame_height,
            "bbox_area": bbox_area,
            "center_x": center_x,
            "center_y": center_y,
            "distance_to_center": distance_to_center,
            **{f"pose_{i}": v for i, v in enumerate(keypoints)}
        })

        frame_index += 1

    cap.release()

# Traverse dataset and extract frame-level features
for root, dirs, files in os.walk(BASE_PATH):
    for file in tqdm(sorted(files)):
        if file.endswith(".mp4"):
            video_path = os.path.join(root, file)
            category = os.path.basename(os.path.dirname(video_path))
            process_video(video_path, category)

# Save frame-level features to CSV
frame_df = pd.DataFrame(frame_level_data)
frame_df.to_csv(CSV_OUTPUT_PATH, index=False)
print(f"✅ Frame-level data saved to: {CSV_OUTPUT_PATH}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 108MB/s]
100%|██████████| 20/20 [04:20<00:00, 13.02s/it]
0it [00:00, ?it/s]


✅ Frame-level data saved to: /content/drive/MyDrive/FINAL DATASET/frame_level_feature_data.csv


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import av
import torch
from torchvision import transforms
from pytorchvideo.models.hub import slowfast_r50

# === PATHS ===
BASE_PATH = "/content/drive/MyDrive/FINAL DATASET"
CSV_OUTPUT_PATH = "/content/drive/MyDrive/FINAL DATASET/video_level_embedding_data.csv"

# === Setup SlowFast Model ===
device = "cuda" if torch.cuda.is_available() else "cpu"
sf_model = slowfast_r50(pretrained=True)
sf_model = torch.nn.Sequential(*list(sf_model.blocks[:-1]))
sf_model.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.45] * 3, [0.225] * 3)
])

def prepare_slowfast_inputs(frames):
    fast = torch.stack([transform(f) for f in frames], dim=1)
    slow = fast[:, ::4, :, :]
    return [slow.unsqueeze(0).to(device), fast.unsqueeze(0).to(device)]

def extract_slowfast_embedding(video_path, num_frames=32):
    try:
        container = av.open(video_path)
        frames = [Image.fromarray(f.to_ndarray(format="rgb24")) for f in container.decode(video=0)]
        if not frames:
            return None
        if len(frames) < num_frames:
            frames += [frames[-1]] * (num_frames - len(frames))
        else:
            idxs = np.linspace(0, len(frames) - 1, num_frames).astype(int)
            frames = [frames[i] for i in idxs]

        inputs = prepare_slowfast_inputs(frames)
        with torch.no_grad():
            features = sf_model(inputs).squeeze().cpu().numpy()
        return features
    except Exception as e:
        print(f"[SlowFast Error] {video_path}: {e}")
        return None

# Extract and store video-level embeddings
embedding_data = []

for root, dirs, files in os.walk(BASE_PATH):
    for file in tqdm(sorted(files)):
        if file.endswith(".mp4"):
            video_path = os.path.join(root, file)
            category = os.path.basename(os.path.dirname(video_path))
            video_name = os.path.basename(video_path)
            features = extract_slowfast_embedding(video_path)
            if features is not None:
                row = {"category": category, "video_name": video_name, **{f"embed_{i}": v for i, v in enumerate(features)}}
                embedding_data.append(row)

# Save to CSV
embedding_df = pd.DataFrame(embedding_data)
embedding_df.to_csv(CSV_OUTPUT_PATH, index=False)
print(f"Video-level embeddings saved to: {CSV_OUTPUT_PATH}")

100%|██████████| 20/20 [00:10<00:00,  1.82it/s]
0it [00:00, ?it/s]


Video-level embeddings saved to: /content/drive/MyDrive/FINAL DATASET/video_level_embedding_data.csv
